# 3.1 Best Practices with faces

-----
Rekognition models used for face comparison operations are designed to work for a wide variety of poses, facial expressions, age ranges, rotations, lighting conditions, and sizes. We recommend that you use the following guidelines when choosing reference photos for CompareFaces or for adding faces to a collection using IndexFaces.


## Guidelines
-----
The following best practices and guidelines for getting the most out of Rekognition for Identity Verification use cases: 


General Pose Guidelines:  

- Use an image with a face that is within the recommended range of angles. The pitch should be less than 30 degrees face down and less than 45 degrees face up. The yaw should be less than 45 degrees in either direction. There is no restriction on the roll.


General Image Quality Guidelines: 

- Use an image of a face with both eyes open and visible.

- Use an image of a face that is not obscured or tightly cropped. The image should contain the full head and shoulders of the person. It should not be cropped to the face bounding box.

- Avoid items that block the face, such as sunglasses and masks.

- Ensure face size is greater than 50x50 pixels

- Use color images.

- Use images with flat lighting on the face, as opposed to varied lighting such as shadows.

- Use images that have sufficient contrast with the background. A high-contrast monochrome background works well.

- Use images of faces with neutral facial expressions with mouth closed and little to no smile for applications that require high precision.

- Use images that are bright and sharp. Avoid using images that may be blurry due to subject and camera motion as much as possible. DetectFaces can be used to determine the brightness and sharpness of a face.

Creating Collections with Multiple Images of an Individual: 

- When creating a collection using IndexFaces, use multiple face images of an individual with different pitches and yaws (within the recommended range of angles). We recommend that at least five images of the person are indexed—straight  on, face turned left with a yaw of 45 degrees or less, face turned right with a yaw of 45 degrees or less, face tilted down with a pitch of 30 degrees or less, and face tilted up with a pitch of 45 degrees or less. If you want to track that these face instances belong to the same individual, consider using the external image ID attribute if there is only one face in the image being indexed. For example, five images of John Doe can be tracked in the collection with external image IDs as John_Doe_1.jpg, … John_Doe_5.jpg.

In [ ]:
import boto3
import json
import io
import cv2

import numpy as np
from PIL import Image, ImageDraw
from matplotlib.pyplot import imshow
from IPython.display import Image as IImage
from skimage.exposure import is_low_contrast


In [ ]:
client=boto3.client('rekognition')

In [ ]:
def image_check (photo):
    
    with open(photo, 'rb') as image:
        response = client.detect_faces(Image={'Bytes': image.read()},Attributes=['ALL'])
    
    
    #perform image check
    image = Image.open(photo)
    imgWidth, imgHeight = image.size 
    faceDetail = response['FaceDetails'][0]
    pitch = faceDetail['Pose']['Pitch']
    yaw = faceDetail['Pose']['Yaw']
    sunglasses = faceDetail['Sunglasses']['Value']
    box = faceDetail['BoundingBox']
    width = imgWidth * box['Width']
    height = imgHeight * box['Height']
    eyesOpen = faceDetail['EyesOpen']['Value']
    emotion = faceDetail['Emotions'][0]['Type']
    emotionConfidence = faceDetail['Emotions'][0]['Confidence']
    mouthOpen = faceDetail['MouthOpen']['Value']
    image_results = ""
    
    if (width < 50) or (height < 50):
        image_results += f'Face dimensions should be > 50x50 pixels. They are {width:.1f} x {height:.1f}\n'
        
    if (abs(yaw) > 45):
        image_results += f'Yaw (side face rotation) should be less than 45 degrees. It is {abs(yaw)} \n'
        
    if ((pitch > 45)):
        image_results += f'Pitch (looking up) should be less than 45 degrees. It is {pitch} \n'
        
    if ((pitch < -30)):
        image_results += f'Pitch (looking down) should be less than 30 degrees. It is {pitch} \n'
               
    if not eyesOpen:
        image_results += f'Eyes are not open \n'
    
    if sunglasses:
        image_results += f'remove sunglasses \n'
    
    if (mouthOpen):
        image_results += f'Mouth should be closed \n'

    # if no entry in image results, then image is compliant
    if not image_results:
        image_results = "Image is compliant"
        
        
    return image_results



In [ ]:
def low_contrast_check (photo):
    image = cv2.imread(photo)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return is_low_contrast(gray, fraction_threshold=0.35)

In [ ]:
%matplotlib inline

def show_faces (photo):
     

    # Load image
    print(f'photo = {photo}')
    
    with open(photo, 'rb') as image:
    
        #Call DetectFaces 
        response = client.detect_faces(Image={'Bytes': image.read()},Attributes=['ALL'])

        image = Image.open(photo)
        imgWidth, imgHeight = image.size  
        draw = ImageDraw.Draw(image)  


        # calculate and display bounding boxes for each detected face       
        print('Detected faces for ' + photo)    
        for faceDetail in response['FaceDetails']:
            print('The detected face is between ' + str(faceDetail['AgeRange']['Low']) 
                  + ' and ' + str(faceDetail['AgeRange']['High']) + ' years old')

            box = faceDetail['BoundingBox']
            left = imgWidth * box['Left']
            top = imgHeight * box['Top']
            width = imgWidth * box['Width']
            height = imgHeight * box['Height']

            # -- uncomment to see bounding box points --
            #print('Left: ' + '{0:.0f}'.format(left))
            #print('Top: ' + '{0:.0f}'.format(top))
            #print('Face Width: ' + "{0:.0f}".format(width))
            #print('Face Height: ' + "{0:.0f}".format(height))

            points = (
                (left,top),
                (left + width, top),
                (left + width, top + height),
                (left , top + height),
                (left, top)

            )
            draw.line(points, fill='#00d400', width=10)

            # Alternatively can draw rectangle. However you can't set line width.
            # draw.rectangle([left,top, left + width, top + height], outline='#00d400') 

        imshow(np.asarray(image))

        return len(response['FaceDetails'])

In [ ]:
path = "./media/"

## Examples of acceptable face profile

### Front view with head and shoulder - Capture 5 images at different face angles

In [ ]:
photo = path + 'photo1.jpeg'
IImage(filename=photo)

In [ ]:
show_faces (photo)

In [ ]:
print(image_check(photo))
result = 'Good contrast' if not low_contrast_check (photo) else 'Low contrast'
print(f'Image has {result}')

### Face turned right with head and shoulder

In [ ]:
photo = path + 'photo3.jpeg'
IImage(filename=photo) 


In [ ]:
print(image_check(photo))
result = 'Good contrast' if not low_contrast_check (photo) else 'Low contrast'
print(f'Image has {result}')

### Face turned left with head and shoulder

In [ ]:
photo = path + 'photo2.jpeg'
IImage(filename=photo)

In [ ]:
print(image_check(photo))
result = 'Good contrast' if not low_contrast_check (photo) else 'Low contrast'
print(f'Image has {result}')

### Face turned up slightly with head and shoulder

In [ ]:
photo = path + 'photo4.jpeg'
IImage(filename=photo)

In [ ]:
print(image_check(photo))
result = 'Good contrast' if not low_contrast_check (photo) else 'Low contrast'
print(f'Image has {result}')

### Face turned down slightly with head and shoulder

In [ ]:
photo = path + 'photo5.jpeg'
IImage(filename=photo)

In [ ]:
print(image_check(photo))
result = 'Good contrast' if not low_contrast_check (photo) else 'Low contrast'
print(f'Image has {result}')

## Following are examples of unacceptable face profile

### Examples with high yaw and pitch

In [ ]:
photo = path + 'photo6.jpeg'

In [ ]:
IImage(filename=photo) 

In [ ]:
print(image_check(photo))
result = 'Good contrast' if not low_contrast_check (photo) else 'Low contrast'
print(f'Image has {result}')


In [ ]:
photo = path + 'photo7.jpeg'

In [ ]:
IImage(filename=photo)

In [ ]:
print(image_check(photo))
result = 'Good contrast' if not low_contrast_check (photo) else 'Low contrast'
print(f'Image has {result}')


In [ ]:
photo = path + 'photo8.jpeg'
IImage(filename=photo) 

In [ ]:
print(image_check(photo))
result = 'Good contrast' if not low_contrast_check (photo) else 'Low contrast'
print(f'Image has {result}')


In [ ]:
photo = path + 'photo9a.jpeg'
IImage(filename=photo) 

In [ ]:
print(image_check(photo))
result = 'Good contrast' if not low_contrast_check (photo) else 'Low contrast'
print(f'Image has {result}')


### Example of less than 50x50 pixels covering the face 

In [ ]:
photo = path + 'photo13.jpeg'
IImage(filename=photo)

In [ ]:
print(image_check(photo))
result = 'Good contrast' if not low_contrast_check (photo) else 'Low contrast'
print(f'Image has {result}')

### Example of occlusions - Avoid sunglasses, masks, headbands, hat

In [ ]:
photo = path + 'photo10.jpeg'
IImage(filename=photo)

In [ ]:
print(image_check(photo))
result = 'Good contrast' if not low_contrast_check (photo) else 'Low contrast'
print(f'Image has {result}')


In [ ]:
photo = path + 'photo11.jpeg'
IImage(filename=photo) 

In [ ]:
print(image_check(photo))
result = 'Good contrast' if not low_contrast_check (photo) else 'Low contrast'
print(f'Image has {result}')


### Example of low contrast image

In [ ]:
photo = path + 'photo12.jpeg'
IImage(photo)

In [ ]:
print(image_check(photo))
result = 'Good contrast' if not low_contrast_check (photo) else 'Low contrast'
print(f'Image has {result}')


## Dealing with PPE & Masks 
------
In the age of COVID-19 it is not uncommon for users to wear masks, as a best practice we recomend using the DetectProtectiveEquipment API here is an example. 



In [ ]:
photo = path + 'ppe-mask.png'
IImage(photo)

In [ ]:
def ppe_image_check (photo):
    
    with open(photo, 'rb') as image:
        response = client.detect_protective_equipment(
            Image={'Bytes': 
                   image.read()},
                   SummarizationAttributes={'MinConfidence': 80,
                                            'RequiredEquipmentTypes': ['FACE_COVER','HEAD_COVER',]}
        )
    
    return response
    
    
  


## Resources

- https://docs.aws.amazon.com/rekognition/latest/dg/recommendations-facial-input-images.html
- https://docs.aws.amazon.com/rekognition/latest/dg/best-practices.html
    